This is  SQL project is done by Group_number_26 .This python codes are written in vscode Jupyter Notebook 

In [ ]:

# Establishing the snowflakes connection  with username, password 
import snowflake.connector
conn = snowflake.connector.connect(
    user='smurali',
    password='SQLproject123@',
    account='rsb78904'
)
cs = conn.cursor()
ware_house_name='warehouse1'
Database_name='Group_26_DataBase'
cs.execute(f"CREATE WAREHOUSE IF NOT EXISTS {ware_house_name}")
cs.execute(f"Create DATABASE IF NOT EXISTS {Database_name}") 

In [90]:
# creating the file formats (csv, xml)

cs.execute("use Database Group_26_DataBase")
query_to_create_csv_format = """
CREATE OR REPLACE FILE FORMAT csv_format
TYPE = 'CSV'
FIELD_OPTIONALLY_ENCLOSED_BY = '"'
"""
cs.execute(query_to_create_csv_format)

# XML format
query_to_create_xml_format = """
CREATE OR REPLACE FILE FORMAT xml_format
TYPE = 'XML'
"""
cs.execute(query_to_create_xml_format)


Part1

Extract and load the 41 comma delimited purchases data files and form a single table of purchases
data

In [91]:
# schema 
Database_name = 'Group_26_DataBase'
cs.execute(f"use DATABASE {Database_name}") 

schema_name= 'purchase_order_Schema'
try:
    cs.execute(f"CREATE SCHEMA IF NOT EXISTS {schema_name} ")
    print("Schema created successfully")

except Exception as e:
    print(f"An error occurred: {e}")


# creating a stage names 
stage_name ='purchase_order_stage'
schema_name= 'purchase_order_Schema'
try:
    cs.execute(f"CREATE OR REPLACE STAGE {schema_name}.{stage_name}")
    print("Stage created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")


Schema created successfully
Stage created successfully.


In [92]:
cs.execute(
    """ 
    CREATE OR REPLACE TABLE PurchaseOrders (
        PurchaseOrderID INT,
        SupplierID INT,
        OrderDate DATE,
        DeliveryMethodID INT,
        ContactPersonID INT,
        ExpectedDeliveryDate DATE,
        SupplierReference VARCHAR(255),
        IsOrderFinalized BOOLEAN,
        Comments VARCHAR,
        InternalComments VARCHAR,
        LastEditedBy INT,
        LastEditedWhen TIMESTAMP,
        PurchaseOrderLineID INT,
        StockItemID INT,
        OrderedOuters INT,
        Description VARCHAR,
        ReceivedOuters INT,
        PackageTypeID INT,
        ExpectedUnitPricePerOuter FLOAT,
        LastReceiptDate DATE,
        IsOrderLineFinalized BOOLEAN,
        Right_LastEditedBy INT,
        Right_LastEditedWhen TIMESTAMP
    );
    """
)


In [93]:
import glob
import os
csv_directory = 'Monthlypurchaseorder/'

print(f"Looking for CSV files in: {os.path.abspath(csv_directory)}")

# Use glob to find all CSV files in the directory
csv_files = glob.glob(os.path.join(csv_directory, "*.csv"))

stage_name ='purchase_order_stage'
schema_name= 'purchase_order_Schema'
cs.execute('use schema purchase_order_Schema')

for file_path in csv_files:
    # The PUT command uploads a file to a stage
    cs.execute(f"PUT file://{file_path} @{schema_name}.{stage_name}")

# Confirm the files were uploaded
cs.execute(f"LIST @{schema_name}.{stage_name}")
for row in cs.fetchall():
    print(row)


Looking for CSV files in: /home/jovyan/Desktop/testrepo/Monthlypurchaseorder
('purchase_order_stage/2019-1.csv.gz', 6784, 'e1ddb7b6b1789c60184dc7e5b51d062a', 'Sun, 1 Sep 2024 22:44:09 GMT')
('purchase_order_stage/2019-10.csv.gz', 3328, 'eb0320ffdb6f4178942e76e17e2be4b6', 'Sun, 1 Sep 2024 22:44:20 GMT')
('purchase_order_stage/2019-11.csv.gz', 3056, '149fded7aceb8ea5fbfa4e32133f1d5d', 'Sun, 1 Sep 2024 22:44:20 GMT')
('purchase_order_stage/2019-12.csv.gz', 2992, 'd000e3e35725f214e50819b595643b79', 'Sun, 1 Sep 2024 22:44:20 GMT')
('purchase_order_stage/2019-2.csv.gz', 2384, 'ec35304584a3b329a35f3fa947d6d83e', 'Sun, 1 Sep 2024 22:44:10 GMT')
('purchase_order_stage/2019-3.csv.gz', 2848, '05ea50ed6085b4b523e5a0bc2f7a39da', 'Sun, 1 Sep 2024 22:44:11 GMT')
('purchase_order_stage/2019-4.csv.gz', 3088, '0f91777e275001aa673ad1318db25538', 'Sun, 1 Sep 2024 22:44:15 GMT')
('purchase_order_stage/2019-5.csv.gz', 3200, '704e8ea3c5d51998844a5670eebe4247', 'Sun, 1 Sep 2024 22:44:16 GMT')
('purchase_order

In [94]:
stage_name ='purchase_order_stage'
schema_name= 'purchase_order_Schema'
Table_name ='PurchaseOrders'


copy_into_command = f"""
COPY INTO {schema_name}.{Table_name}
FROM @purchase_order_Schema.{stage_name}
FILE_FORMAT = (
TYPE = 'CSV'
FIELD_OPTIONALLY_ENCLOSED_BY = '"'
SKIP_HEADER=1
TIMESTAMP_FORMAT = 'MM/DD/YYYY HH:MI'
DATE_FORMAT = 'MM/DD/YYYY'
NULL_IF = ('NULL', '00:00.0', '2/29/2022', '2/28/2022 7:00:00 AM')
)
ON_ERROR = 'CONTINUE';
"""

# Execute the COPY INTO command and capture the output
try:
    cs.execute(copy_into_command)
    print("Data load completed.")
except Exception as e:
    print(f"Error during COPY INTO: {e}")

# Verify the number of rows in the table
cs.execute(f"SELECT COUNT(*) FROM {schema_name}.{Table_name}")
count = cs.fetchone()[0]
print(f"Total rows in {Table_name}: {count}")

#verifying the columns

cs.execute(f"""
    SELECT COUNT(*)
    FROM information_schema.columns
    WHERE table_schema = '{schema_name.upper()}'
    AND table_name = '{Table_name.upper()}';
""")
column_count = cs.fetchone()[0]
print(f"Total columns in {Table_name}: {column_count}")



Data load completed.
Total rows in PurchaseOrders: 8357
Total columns in PurchaseOrders: 23


Part 2

Create a calculated field that shows purchase order totals, i.e., for each order, sum the line item
amounts (defined as ReceivedOuters * ExpectedUnitPricePerOuter), and name this field POAmounT

In [95]:
schema_name= 'purchase_order_Schema'
Table_name ='PurchaseOrders'

query_to_alter = f'ALTER TABLE {schema_name}.{Table_name} ADD COLUMN POAmount DECIMAL(18, 2)'
cs.execute(query_to_alter)

cs.execute(f"""
UPDATE {schema_name}.{Table_name}
SET POAmount = ReceivedOuters * ExpectedUnitPricePerOuter;
""")
print("POAmount column updated with calculated values.")
cs.execute(f"""
    SELECT COUNT(*)
    FROM information_schema.columns
    WHERE table_schema = '{schema_name.upper()}'
    AND table_name = '{Table_name.upper()}';
""")
column_count = cs.fetchone()[0]
print(f"Total columns in {Table_name}: {column_count}")


POAmount column updated with calculated values.
Total columns in PurchaseOrders: 24


part 3 

Extract and load the supplier invoice XML data
shred the data into a table (preferably in the COPY INTO process) where each row corresponds to a single invoice

In [96]:

# schema for the XML file  supplierTransactionXML 
schema_name2= 'SupplierTransaction_Schema'
try:
    cs.execute(f"CREATE SCHEMA IF NOT EXISTS {schema_name2} ")
    print("Schema created successfully")

except Exception as e:
    print(f"An error occurred: {e}")



Schema created successfully


In [97]:
# creating a stage names  Xml_stage
schema_name2= 'SupplierTransaction_Schema'
stage_name2='Xml_stage'
try:
    cs.execute(f"CREATE OR REPLACE STAGE {schema_name2}.{stage_name2}")
    print("Stage created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")


Stage created successfully.


In [98]:
#uploading the files into the stage created  using put command, i have the data file inside testdatafolder/SupplierTransactionsXML.xml
schema_name2= 'SupplierTransaction_Schema'
file_path = 'testdata/SupplierTransactionsXML.xml'
stage_name2='Xml_stage'


put_command = f"PUT file://{file_path} @{schema_name2}.{stage_name2}"
print(put_command)
# Execute the PUT command
cs.execute(put_command)



PUT file://testdata/SupplierTransactionsXML.xml @SupplierTransaction_Schema.Xml_stage


In [99]:
# creating a staging table in snowflakes with a variant xml_data

create_table_command = """CREATE OR REPLACE TABLE SupplierTransactions(xml_data VARIANT NOT NULL);"""
cs.execute(create_table_command)

# COPY INTO command from the copying the xml file to the staging table
stage_name2 ='Xml_stage'
schema_name2= 'SupplierTransaction_Schema'
copy_into_command = f""" COPY INTO SupplierTransactions FROM @{schema_name2}.{stage_name2} FILE_FORMAT = (TYPE = 'XML') ON_ERROR='CONTINUE';"""

try:
    # Execute the COPY INTO command
    cs.execute(copy_into_command)
    print("Data copied into the  staging table successfully")

except Exception as e:
    print(f"An error occurred while copying data into the table: {e}")


Data copied into the  staging table successfully


In [100]:

schema_name2= 'SupplierTransaction_Schema'
Table_name ='SupplierTransactions'

cs.execute(f"""
CREATE OR REPLACE TABLE {schema_name2}.{Table_name} AS
SELECT
    XMLGET( r.value, 'SupplierTransactionID' ):"$"::INTEGER AS SupplierTransactionID,
    XMLGET( r.value, 'SupplierID' ):"$"::INTEGER AS SupplierID,
    XMLGET( r.value, 'TransactionTypeID' ):"$"::INTEGER AS TransactionTypeID,
    CASE WHEN REGEXP_LIKE(XMLGET(r.value, 'PurchaseOrderID'):"$", '^[0-9]+$') THEN TO_NUMBER(XMLGET(r.value, 'PurchaseOrderID'):"$") ELSE NULL END AS PurchaseOrderID,
    XMLGET( r.value, 'PaymentMethodID' ):"$"::INTEGER AS PaymentMethodID,
    CASE WHEN REGEXP_LIKE(XMLGET(r.value, 'SupplierInvoiceNumber'):"$", '^[0-9]+$') THEN TO_NUMBER(XMLGET(r.value, 'SupplierInvoiceNumber'):"$") ELSE NULL END AS SupplierInvoiceNumber,
    XMLGET( r.value, 'TransactionDate' ):"$"::STRING AS TransactionDate,
    XMLGET( r.value, 'AmountExcludingTax' ):"$"::FLOAT AS AmountExcludingTax,
    XMLGET( r.value, 'TaxAmount' ):"$"::FLOAT AS TaxAmount,
    XMLGET( r.value, 'TransactionAmount' ):"$"::FLOAT AS TransactionAmount,
    XMLGET( r.value, 'OutstandingBalance' ):"$"::FLOAT AS OutstandingBalance,
    XMLGET( r.value, 'FinalizationDate' ):"$"::STRING AS FinalizationDate,
    XMLGET( r.value, 'IsFinalized' ):"$"::INTEGER AS IsFinalized,
    XMLGET( r.value, 'LastEditedBy' ):"$"::INTEGER AS LastEditedBy,
    XMLGET( r.value, 'LastEditedWhen' ):"$"::STRING AS LastEditedWhen
FROM SupplierTransactions,  
LATERAL FLATTEN(SupplierTransactions.xml_data:"$") r
WHERE GET( r.value, '@') = 'row' """)
#verifying the columns
cs.execute(f"""
    SELECT COUNT(*)
    FROM information_schema.columns
    WHERE table_schema = '{schema_name2.upper()}'
    AND table_name = '{Table_name.upper()}';
""")
column_count = cs.fetchone()[0]
print(f"Total columns in {Table_name}: {column_count}")

Total columns in SupplierTransactions: 15


Part 4

Join the purchases data from step 2 and the supplier invoices data from step 3 (only include
matching rows);

In [101]:

command = """
CREATE OR REPLACE TABLE PurchaseOrderSupplierTransaction AS
SELECT a.PurchaseOrderID, 
a.SupplierID, a. OrderDate, a.ExpectedDeliveryDate, a.PurchaseOrderLineID, a.StockItemID, 
a.OrderedOuters, a.ReceivedOuters, a.ExpectedUnitPricePerOuter , a.LastReceiptDate, a.POAmount, b.SupplierTransactionID, b.SupplierInvoiceNumber, 
b.TransactionDate, b.AmountExcludingTax, b.TaxAmount, b.TransactionAmount, b.FinalizationDate
FROM purchase_order_Schema.PurchaseOrders a
JOIN SupplierTransaction_Schema.SupplierTransactions b 
using (SupplierID,PurchaseOrderID)
"""
cs.execute(command)


Part 5

Using the joined data from step 4, create a calculated field that shows the difference between
AmountExcludingTax and POAmount, name this field invoiced_vs_quoted, and save the result as a
materialized view named purchase_orders_and_invoices

In [102]:
query_to_create_view = """
CREATE OR REPLACE MATERIALIZED VIEW PURCHASE_ORDERS_AND_INVOICES AS (
SELECT AmountExcludingTax, POAmount, (AmountExcludingTax - POAmount) AS invoiced_vs_quoted
FROM PurchaseOrderSupplierTransaction
)
"""
cs.execute(query_to_create_view)

In [103]:
cs.execute("select * from PURCHASE_ORDERS_AND_INVOICES order by invoiced_vs_quoted desc limit 10")
rows = cs.fetchmany(10)
for row in rows:
    print(rows)

[(1627734.0, Decimal('153300.00'), 1474434.0), (1624638.0, Decimal('153048.00'), 1471590.0), (1619580.0, Decimal('152376.00'), 1467204.0), (1618188.0, Decimal('152208.00'), 1465980.0), (1618332.0, Decimal('152544.00'), 1465788.0), (1616604.0, Decimal('152544.00'), 1464060.0), (1616370.0, Decimal('152880.00'), 1463490.0), (1627734.0, Decimal('164790.00'), 1462944.0), (1613106.0, Decimal('150780.00'), 1462326.0), (1624638.0, Decimal('164430.00'), 1460208.0)]
[(1627734.0, Decimal('153300.00'), 1474434.0), (1624638.0, Decimal('153048.00'), 1471590.0), (1619580.0, Decimal('152376.00'), 1467204.0), (1618188.0, Decimal('152208.00'), 1465980.0), (1618332.0, Decimal('152544.00'), 1465788.0), (1616604.0, Decimal('152544.00'), 1464060.0), (1616370.0, Decimal('152880.00'), 1463490.0), (1627734.0, Decimal('164790.00'), 1462944.0), (1613106.0, Decimal('150780.00'), 1462326.0), (1624638.0, Decimal('164430.00'), 1460208.0)]
[(1627734.0, Decimal('153300.00'), 1474434.0), (1624638.0, Decimal('153048.00'

PART 6 




Extract the supplier_case data from postgres, do not import the data into Python, instead use
Python to move the data from postgres to your local drive and then directly into a Snowflake stage
a. Consider creating a Python function that can take a csv file path as input and then generate field
definitions (field names and datatypes based on the header and data types in the file) that can
then be used in CREATE TABLE statement.
b. You need to use psycopg2 or a similar Python library to connect to the postgres database within
Python, issue a command to postgres to have postgres save the supplier_case data to file, and
then use cs.execute to move the file to an internal Snowflake stage and eventually into a table.

In [104]:
#specifying the Database , Schema for supplier data, Stage 
Database_name = 'Group_26_DataBase'
cs.execute(f"use DATABASE {Database_name}") 

stage_name3 = "Supplier"
schema_name3= 'Supplier_schema'

cs.execute(f"CREATE SCHEMA IF NOT EXISTS {schema_name3} ")
cs.execute(f"CREATE OR REPLACE STAGE {schema_name3}.{stage_name3} ")

In [105]:
import psycopg2
import os

def export_data_to_csv(postgres_conn_details, query, output_csv_path):
    """
    Export data from a PostgreSQL database to a CSV file 
    """
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(**postgres_conn_details)
        cursor = conn.cursor()

        # Write the data to a CSV file using COPY TO STDOUT
        with open(output_csv_path, 'w') as csvfile:
            cursor.copy_expert(query, csvfile)

        print(f"Data successfully exported to {output_csv_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Define PostgreSQL connection details
postgres_conn_details = {
    'dbname': 'WestCoastImporters',   
    'user': 'jovyan',   
    'password': 'postgres',   
    'host': '127.0.0.1',  
    'port': 8765 
}

# Define the SQL COPY query
query = "COPY (SELECT * FROM supplier_case) TO STDOUT WITH CSV HEADER"

# Get the current working directory
directory = os.getcwd()

# Check if the directory exists, if not, create it
if not os.path.exists(directory):
    os.makedirs(directory)

csv_file_path = os.path.join(directory, 'supplier_case.csv')

# function call
export_data_to_csv(postgres_conn_details, query, csv_file_path)


Data successfully exported to /home/jovyan/Desktop/testrepo/supplier_case.csv


In [106]:
# load the csv in to the stage

stage_name3 = "Supplier"
schema_name3= 'Supplier_schema'
try:

    csv_file = os.getcwd()+'/supplier_case.csv'
    print(csv_file)
    cs.execute(f"PUT file://{csv_file} @{schema_name3}.{stage_name3}")
    print(f"File {csv_file} successfully uploaded to Snowflake stage @{stage_name3}.")

except Exception as e:
        print(f"An error occurred: {e}")


/home/jovyan/Desktop/testrepo/supplier_case.csv
File /home/jovyan/Desktop/testrepo/supplier_case.csv successfully uploaded to Snowflake stage @Supplier.


In [107]:
command = """
CREATE OR REPLACE FILE FORMAT SUPPLIER_CASE 
    TYPE = CSV
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    PARSE_HEADER = TRUE
    TIMESTAMP_FORMAT = 'MM/DD/YYYY HH24:MI'
"""
cs.execute(command)


In [108]:

sql_table="""
    CREATE OR REPLACE TABLE Supplier_schema.supplierCase (
        SupplierID INTEGER,
        SupplierName STRING,
        SupplierCategoryID INTEGER,
        PrimaryContactPersonID INTEGER,
        alternatecontactpersonid INTEGER,
        DeliveryMethodID INTEGER,
        PostalCityID INTEGER,
        SupplierReference STRING,
        BankAccountName STRING,
        BankAccountBranch STRING,
        BankAccountCode INTEGER,
        BankAccountNumber INTEGER,
        BankInternationalCode INTEGER,
        PaymentDays INTEGER,
        InternalComments STRING,
        PhoneNumber STRING,
        FaxNumber STRING,
        WebsiteURL STRING,
        DeliveryAddressLine1 STRING,
        DeliveryAddressLine2 STRING,
        DeliveryPostalCode STRING,
        DeliveryLocation STRING,
        PostalAddressLine1 STRING,
        PostalAddressLine2 STRING,
        PostalPostalCode INTEGER,
        Lasteditedby INTEGER,
        ValidFrom STRING,
        ValidTo STRING
    );
"""
cs.execute(sql_table)


In [109]:
#Check what is csv name inside snowflake stage
cs.execute(f"LIST @{schema_name3}.{stage_name3};")
staged_files = cs.fetchall()
for file in staged_files:
    print(file)

('supplier/supplier_case.csv.gz', 1968, '007238f66fde17b20daf6a014ac70416', 'Sun, 1 Sep 2024 22:44:35 GMT')


In [110]:
stage_name3 = "Supplier"
schema_name3= 'Supplier_schema'
cs.execute(
    """
    COPY INTO Supplier_schema.supplierCase
    FROM @Supplier_schema.Supplier
    FILE_FORMAT = SUPPLIER_CASE
    MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
    ON_ERROR = 'CONTINUE' 
    """
)
# Fetch and display any output or errors
load_results = cs.fetchall()
for result in load_results:
    print(result)

('supplier/supplier_case.csv.gz', 'LOADED', 13, 13, 13, 0, None, None, None, None)


Part 7 a
While the weather station data contain zip codes, we will pretend that this table does not have this information and instead use latitude and longitude information to determine which weather station to use for each zip code. The approach used in
https://towardsdatascience.com/noaa-weather-data-in-snowflake-free-20e90ee916ed 
can be helpful for (note that this is based on a different data set, but the idea of using latitude and longitude is the same) 
finding weather stations closest to each zip code (only use one weather station per zip code). 
For this to work you need to find a data file with zip code – geo location mappings, e.g., from the US census (the
data zip folder on Canvas contains a ZCTA file with this information; in this file GEOID is the five
digit ZIP Code, INTPTLAT is Latitude, and INTPTLONG is Longitude)

In [111]:
#specifying the Database , Schema for supplier data, Stage 
Database_name = 'Group_26_DataBase'
cs.execute(f"use DATABASE {Database_name}") 

stage_name4 = "weather_stage"
schema_name4= 'weather_schema'

cs.execute(f"CREATE SCHEMA IF NOT EXISTS {schema_name4} ")
cs.execute(f"CREATE OR REPLACE STAGE {schema_name4}.{stage_name4} ")

In [112]:
# getting the data from the local and uploading it to the stage

import os
file = os.getcwd() + "/testdata/2021_Gaz_zcta_national.txt"
stage_name4 = "weather_stage"
schema_name4= 'weather_schema' 
print(file)
cs.execute(f"PUT file://{file} @weather_schema.weather_stage")

/home/jovyan/Desktop/testrepo/testdata/2021_Gaz_zcta_national.txt


In [113]:
# creating a table name zcta_table
cs.execute("""CREATE OR REPLACE TABLE zcta_table (
    GEOID VARCHAR(5),
    ALAND NUMERIC,
    AWATER NUMERIC,
    ALAND_SQMI FLOAT,
    AWATER_SQMI FLOAT,
    INTPTLAT FLOAT,
    INTPTLONG FLOAT
);
""")

In [114]:
# copy data in to the table from the stage
cs.execute("""
COPY INTO zcta_table
FROM @weather_schema.weather_stage
FILE_FORMAT = (
    TYPE = 'CSV',
    FIELD_OPTIONALLY_ENCLOSED_BY = '"',
    FIELD_DELIMITER = '\t',
    SKIP_HEADER = 1
)""")

In [115]:
# joining two tables and creating a table with  required columns
cs.execute("""
        CREATE OR REPLACE TABLE supplier_zip_geolocation AS (
        SELECT sc.PostalPostalCode, z.INTPTLAT AS latitude, z.INTPTLONG AS longitude
        FROM Supplier_schema.supplierCase sc
        JOIN zcta_table z ON sc.postalpostalcode = z.GEOID
        GROUP BY sc.PostalPostalCode, z.INTPTLAT, z.INTPTLONG);
""")

In [116]:
# creating the Table zip_weather_station_mapping 
# by cross joining the supplier_zip_geolocation,JOIN WEATHER__ENVIRONMENT.cybersyn.noaa_weather_station_index 

cs.execute("""
CREATE OR REPLACE TABLE zip_weather_station_mapping AS
    SELECT 
        sz.PostalPostalCode,
        ws.noaa_weather_station_id AS station_id,
        ws.noaa_weather_station_name AS station_name,
        SQRT(
            POW(69.1 * (sz.latitude - ws.latitude), 2) +
            POW(69.1 * (ws.longitude - sz.longitude) * COS(sz.latitude / 57.3), 2)
        ) AS distance
    FROM supplier_zip_geolocation sz
    CROSS JOIN WEATHER__ENVIRONMENT.cybersyn.noaa_weather_station_index ws
    QUALIFY ROW_NUMBER() OVER (PARTITION BY sz.postalpostalcode ORDER BY distance ASC) = 1;
""")

In [117]:
#creating a table zip_weather_station_mapping_temp 
cs.execute("""
CREATE OR REPLACE TABLE zip_weather_station_mapping_temp AS
    SELECT 
        sz.postalpostalcode,
        ws.noaa_weather_station_id AS station_id,
        ws.noaa_weather_station_name AS station_name,
        st_distance(
            st_makepoint(sz.longitude, sz.latitude),
            st_makepoint(ws.longitude, ws.latitude)
        ) AS distance
    FROM supplier_zip_geolocation sz
    JOIN WEATHER__ENVIRONMENT.cybersyn.noaa_weather_station_index ws
    ON st_distance(st_makepoint(sz.longitude, sz.latitude), st_makepoint(ws.longitude, ws.latitude)) < 50000
    QUALIFY ROW_NUMBER() OVER (PARTITION BY sz.postalpostalcode ORDER BY distance ASC) = 1;
""")

Part 7 b Create a materialized view named supplier_zip_code_weather that contains the unique zip codes (PostalPostalCode) from the supplier data, date, and daily high temperatures, i.e., the view should have three columns (zip code, date, and high temperature) and one row per day and unique supplier zip code

In [118]:
cs.execute("""
CREATE OR REPLACE TABLE zip_code_weather_data AS
           SELECT 
            z.postalpostalcode,
            wm.date,
            wm.value as Max_Temp,
            wm.variable
            FROM zip_weather_station_mapping z
            JOIN WEATHER__ENVIRONMENT.cybersyn.noaa_weather_metrics_timeseries wm
            ON z.station_id = wm.noaa_weather_station_id
            WHERE wm.variable = 'maximum_temperature';
""")

In [119]:
cs.execute("""
CREATE OR REPLACE TABLE zip_code_weather_data_temp AS
           SELECT 
            z.postalpostalcode,
            wm.date,
            wm.value as Max_Temp,
            wm.variable
            FROM zip_weather_station_mapping_temp z
            JOIN WEATHER__ENVIRONMENT.cybersyn.noaa_weather_metrics_timeseries wm
            ON z.station_id = wm.noaa_weather_station_id
            WHERE wm.variable = 'maximum_temperature';
""")

In [120]:
cs.execute("""
CREATE OR REPLACE TABLE supplier_zip_code_weather_temp AS 
SELECT A.PostalPostalCode, B.date, B.Max_temp 
        FROM Supplier_schema.supplierCase as A
        LEFT JOIN zip_code_weather_data as B ON A.postalpostalcode = B.postalpostalcode
        ORDER BY DATE;
""")

In [124]:
# creating a materialized view 

command = """CREATE OR REPLACE MATERIALIZED VIEW supplier_zip_code_weather AS 
            (SELECT * FROM supplier_zip_code_weather_temp);"""
cs.execute(command)


PART 8 

Join purchase_orders_and_invoices, supplier_case, and supplier_zip_code_weather based on zip
codes and the transaction date. Only include transactions that have matching temperature readings

In [ ]:

cs.execute("""CREATE OR REPLACE VIEW order_zip_temp AS 
SELECT A.*
FROM supplier_zip_code_weather A
LEFT JOIN SupplierTransaction_Schema.PurchaseOrderSupplierTransaction B 
ON A.DATE = B.TRANSACTIONDATE
LEFT JOIN Supplier_schema.supplierCase C 
ON A.postalpostalcode = C.postalpostalcode
WHERE A.max_temp IS NOT NULL;""")
